In [49]:
from pathlib import Path

OUTPUT_DIR = DATA_DIR
print("Outputs will be saved in:", OUTPUT_DIR.resolve())

DATA_DIR = Path(r"C:\Users\patri\Desktop\Folders\EU Classes\10) 691_DTSC Applied Data Science\Challenges\Challenge 1 Data")
assert DATA_DIR.exists(), f"Folder not found: {DATA_DIR}"

expected = {
    "Inventory_Snapshots_2024.csv",
    "Products_2024.csv",
    "Promotions_2024.csv",
    "Sales_2024_Q1.csv",
    "Sales_2024_Q2.csv",
    "Sales_2024_Q3.csv",
    "Sales_2024_Q4.csv",
}

present = {p.name for p in DATA_DIR.glob("*.csv")}
missing = expected - present
extra = present - expected

print("Data folder:", DATA_DIR, "\n")
print("All files in folder:")
for p in DATA_DIR.iterdir():
    print(" -", p.name)

print("\nCSV files found:", sorted(present))
print("Missing files:", sorted(missing) if missing else "None ✅")
print("Unexpected extras:", sorted(extra) if extra else "None")

Outputs will be saved in: C:\Users\patri\Desktop\Folders\EU Classes\10) 691_DTSC Applied Data Science\Challenges\Challenge 1 Data
Data folder: C:\Users\patri\Desktop\Folders\EU Classes\10) 691_DTSC Applied Data Science\Challenges\Challenge 1 Data 

All files in folder:
 - Inventory_Snapshots_2024.csv
 - Products_2024.csv
 - Promotions_2024.csv
 - README.txt
 - Sales_2024_Q1.csv
 - Sales_2024_Q2.csv
 - Sales_2024_Q3.csv
 - Sales_2024_Q4.csv

CSV files found: ['Inventory_Snapshots_2024.csv', 'Products_2024.csv', 'Promotions_2024.csv', 'Sales_2024_Q1.csv', 'Sales_2024_Q2.csv', 'Sales_2024_Q3.csv', 'Sales_2024_Q4.csv']
Missing files: None ✅
Unexpected extras: None


In [50]:
import pandas as pd

products = pd.read_csv(DATA_DIR / "Products_2024.csv", low_memory=False)

for c in ["product_id", "category_id", "vendor_id"]:
    if c in products.columns:
        products[c] = pd.to_numeric(products[c], errors="coerce").astype("Int64")

print("Products shape:", products.shape)
display(products.head(5))
print("\nDtypes:\n", products.dtypes)

if "product_id" in products.columns:
    dup_count = products.duplicated(subset=["product_id"]).sum()
    print(f"\nDuplicate product_id rows: {dup_count}")
else:
    print("\nNote: 'product_id' column not found — we'll confirm the schema in the next step.")

Products shape: (60, 12)


,product_upc,product_name,brand,department_name,category_name,size,unit,vendor_name,vendor_phone,regular_price,unit_cost,pack_size
0,433218196001,Rainshadow Produce Apples,Rainshadow Produce,Produce,Apples,1 lb,lb,Evergreen Organics,206-555-0110,3.11,2.31,1
1,386379402654,Evergreen Farms Bananas,Evergreen Farms,Produce,Bananas,1 lb,lb,Frontier Foods Wholesale,206-555-0118,5.33,3.68,1
2,161559407816,Evergreen Farms Kale,Evergreen Farms,Produce,Kale,3 lb,lb,Pacific Produce Co.,206-555-0111,1.76,1.10,1
3,931034131647,Rainshadow Produce Spinach,Rainshadow Produce,Produce,Spinach,3 lb,lb,Bulk Barn West,206-555-0115,1.15,0.84,6
4,534192832764,North Fork Organics Carrots,North Fork Organics,Produce,Carrots,2 lb,lb,Bulk Barn West,206-555-0115,4.52,3.19,6



Dtypes:
 product_upc          int64
product_name        object
brand               object
department_name     object
category_name       object
size                object
unit                object
vendor_name         object
vendor_phone        object
regular_price      float64
unit_cost          float64
pack_size            int64
dtype: object

Note: 'product_id' column not found — we'll confirm the schema in the next step.


In [51]:
promotions = pd.read_csv(DATA_DIR / "Promotions_2024.csv", low_memory=False)

for c in ["start_date", "end_date"]:
    if c in promotions.columns:
        promotions[c] = pd.to_datetime(promotions[c], errors="coerce")

for c in ["promo_id", "category_id", "store_id"]:
    if c in promotions.columns:
        promotions[c] = pd.to_numeric(promotions[c], errors="coerce").astype("Int64")

print("Promotions shape:", promotions.shape)
display(promotions.head(5))
print("\nDtypes:\n", promotions.dtypes)

if {"start_date", "end_date"}.issubset(promotions.columns):
    bad_order = (promotions["start_date"] > promotions["end_date"]).sum()
    print(f"\nRows where start_date > end_date: {bad_order}")

if "discount_pct" in promotions.columns:
    out_of_range = ~((promotions["discount_pct"].between(0, 1)) | (promotions["discount_pct"].between(0, 100)))
    print(f"Rows with discount_pct outside [0–1] or [0–100]: {out_of_range.sum()}")

Promotions shape: (530, 9)


,promo_id,product_upc,promo_type,discount_percent,start_date,end_date,product_name,department_name,brand
0,<NA>,471349361832,PCT_OFF,15,2024-01-01,2024-01-07,Meadow Meats Salmon Fillet,Meat/Seafood,Meadow Meats
1,<NA>,831727889579,LOYALTY_PCT,30,2024-01-01,2024-01-07,Sun Valley Tortillas,Grocery,Sun Valley
2,<NA>,190229413186,LOYALTY_PCT,5,2024-01-01,2024-01-07,Greenway Kitchen Tomato Soup,Deli,Greenway Kitchen
3,<NA>,452991241904,PCT_OFF,30,2024-01-01,2024-01-07,Sunrise Labs Magnesium,Wellness,Sunrise Labs
4,<NA>,216073375433,BOGO50,50,2024-01-01,2024-01-07,Sun Valley Basmati Rice,Grocery,Sun Valley



Dtypes:
 promo_id                     Int64
product_upc                  int64
promo_type                  object
discount_percent             int64
start_date          datetime64[ns]
end_date            datetime64[ns]
product_name                object
department_name             object
brand                       object
dtype: object

Rows where start_date > end_date: 0


In [52]:
import pandas as pd

def load_sales_csv(path):
    df = pd.read_csv(path, low_memory=False)
    # Normalize date column name
    if "txn_date" not in df.columns and "date" in df.columns:
        df = df.rename(columns={"date": "txn_date"})
    # Parse dates if present
    if "txn_date" in df.columns:
        df["txn_date"] = pd.to_datetime(df["txn_date"], errors="coerce")

    for c, kind in {
        "txn_id": "Int64",
        "store_id": "Int64",
        "product_upc": "Int64",
        "product_id": "Int64",
        "qty": "Int64",
    }.items():
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype(kind)
    for c in ["unit_price", "line_amount"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

sales_q1 = load_sales_csv(DATA_DIR / "Sales_2024_Q1.csv")
sales_q2 = load_sales_csv(DATA_DIR / "Sales_2024_Q2.csv")
sales_q3 = load_sales_csv(DATA_DIR / "Sales_2024_Q3.csv")
sales_q4 = load_sales_csv(DATA_DIR / "Sales_2024_Q4.csv")

sales = pd.concat([sales_q1, sales_q2, sales_q3, sales_q4], ignore_index=True)

print("Sales shapes:", { "Q1": sales_q1.shape, "Q2": sales_q2.shape, "Q3": sales_q3.shape, "Q4": sales_q4.shape })
print("Sales (all):", sales.shape)
display(sales.head(5))
print("\nDtypes:\n", sales.dtypes)

neg_cols = [c for c in ["qty", "unit_price", "line_amount"] if c in sales.columns]
if neg_cols:
    neg_counts = {c: int((sales[c] < 0).sum()) for c in neg_cols}
    print("\nNegative values:", neg_counts)

if "txn_id" in sales.columns:
    dup_txn = int(sales.duplicated(subset=["txn_id"]).sum())
    print("Duplicate txn_id rows:", dup_txn)
else:
    subset = [c for c in ["store_id", "product_upc", "product_id", "txn_date"] if c in sales.columns]
    if subset:
        dup_combo = int(sales.duplicated(subset=subset).sum())
        print(f"Duplicate rows by {subset}:", dup_combo)

if all(c in sales.columns for c in ["qty", "unit_price", "line_amount"]):
    calc = sales["qty"] * sales["unit_price"]
    rel_err = (sales["line_amount"] - calc).abs() / calc.replace(0, pd.NA)
    bad = int((rel_err.fillna(0) > 0.01).sum())
    print("Rows where line_amount differs from qty*unit_price by >1%:", bad)

Sales shapes: {'Q1': (13545, 31), 'Q2': (13479, 31), 'Q3': (13097, 31), 'Q4': (13165, 31)}
Sales (all): (53286, 31)


,receipt_id,line_number,sale_datetime,store_id,store_name,store_address,store_city,store_state,store_zip,cashier_name,...,pack_size,regular_price,unit_cost,quantity,promo_type,promo_id,unit_price_effective,line_subtotal,tax_amount,weekend_flag
0,101-20240101-0001,1,01/01/2024 14:20,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,Quinn,...,1,5.34,3.24,1,NaN,NaN,5.34,5.34,0.0,False
1,101-20240101-0001,2,01/01/2024 14:20,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,Quinn,...,6,1.15,0.84,1,LOYALTY_PCT,PR0011647,1.09,1.09,0.0,False
2,101-20240101-0001,3,01/01/2024 14:20,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,Quinn,...,6,4.52,3.19,2,NaN,NaN,4.52,9.04,0.0,False
3,101-20240101-0001,4,01/01/2024 14:20,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,Quinn,...,2,1.09,0.79,2,NaN,NaN,1.09,2.18,0.0,False
4,101-20240101-0002,1,01/01/2024 13:30,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,Riley,...,6,16.74,10.94,2,NaN,NaN,16.74,33.48,0.0,False



Dtypes:
 receipt_id               object
line_number               int64
sale_datetime            object
store_id                  Int64
store_name               object
store_address            object
store_city               object
store_state              object
store_zip                 int64
cashier_name             object
tender_type              object
customer_segment         object
product_upc               Int64
product_name             object
brand                    object
department_name          object
category_name            object
size                     object
unit                     object
vendor_name              object
vendor_phone             object
pack_size                 int64
regular_price           float64
unit_cost               float64
quantity                  int64
promo_type               object
promo_id                 object
unit_price_effective    float64
line_subtotal           float64
tax_amount              float64
weekend_flag               boo

In [53]:
# STEP 5 — Load Inventory Snapshots, parse dates, and run quick checks

import pandas as pd

inv = pd.read_csv(DATA_DIR / "Inventory_Snapshots_2024.csv", low_memory=False)

# Normalize date column name and parse to datetime
if "snapshot_date" not in inv.columns:
    for alt in ["as_of_date", "date"]:
        if alt in inv.columns:
            inv = inv.rename(columns={alt: "snapshot_date"})
            break
inv["snapshot_date"] = pd.to_datetime(inv["snapshot_date"], errors="coerce")

# Coerce common fields to numeric
for c, kind in {"store_id": "Int64", "product_upc": "Int64", "qty_on_hand": "Int64"}.items():
    if c in inv.columns:
        inv[c] = pd.to_numeric(inv[c], errors="coerce").astype(kind)

print("Inventory shape:", inv.shape)
display(inv.head(5))
print("\nDtypes:\n", inv.dtypes)

# --------- Quick integrity checks ---------
alerts = []

# 1) Non-negative quantities
if "qty_on_hand" in inv.columns:
    neg = int((inv["qty_on_hand"] < 0).sum())
    alerts.append(("Negative qty_on_hand", neg))

# 2) Duplicates by (store_id, product_upc, snapshot_date)
subset = [c for c in ["store_id", "product_upc", "snapshot_date"] if c in inv.columns]
if len(subset) == 3:
    dup = int(inv.duplicated(subset=subset).sum())
    alerts.append((f"Duplicate rows by {subset}", dup))

# 3) Dates not in 2024 (adjust if needed)
if "snapshot_date" in inv.columns:
    not_2024 = int(inv["snapshot_date"].dropna().map(lambda d: d.year != 2024).sum())
    alerts.append(("Rows with snapshot_date not in 2024", not_2024))

# 4) Orphan products (inventory product_upc not found in products)
if "product_upc" in inv.columns and "product_upc" in globals()["products"].columns:
    orphan = int(~inv["product_upc"].isin(products["product_upc"]).sum())
    # whoops: the above counts matches; fix:
    orphan = int((~inv["product_upc"].isin(products["product_upc"])).sum())
    alerts.append(("Inventory product_upc not found in Products", orphan))

print("\nAlerts:")
for name, count in alerts:
    print(f" - {name}: {count}")

Inventory shape: (2160, 13)


,snapshot_date,store_id,store_name,store_address,store_city,store_state,store_zip,product_upc,product_name,department_name,on_hand_qty,unit_cost,inventory_cost_value
0,2024-01-01,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,433218196001,Rainshadow Produce Apples,Produce,35,2.31,80.85
1,2024-01-01,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,386379402654,Evergreen Farms Bananas,Produce,43,3.68,158.24
2,2024-01-01,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,161559407816,Evergreen Farms Kale,Produce,42,1.10,46.20
3,2024-01-01,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,931034131647,Rainshadow Produce Spinach,Produce,20,0.84,16.80
4,2024-01-01,101,Greenway - Northside,1201 Maple Ave,Everton,WA,98111,534192832764,North Fork Organics Carrots,Produce,55,3.19,175.45



Dtypes:
 snapshot_date           datetime64[ns]
store_id                         Int64
store_name                      object
store_address                   object
store_city                      object
store_state                     object
store_zip                        int64
product_upc                      Int64
product_name                    object
department_name                 object
on_hand_qty                      int64
unit_cost                      float64
inventory_cost_value           float64
dtype: object

Alerts:
 - Duplicate rows by ['store_id', 'product_upc', 'snapshot_date']: 0
 - Rows with snapshot_date not in 2024: 0
 - Inventory product_upc not found in Products: 0


In [54]:
tables = {
    "Products": products,
    "Promotions": promotions,
    "Sales_Q1": sales_q1,
    "Sales_Q2": sales_q2,
    "Sales_Q3": sales_q3,
    "Sales_Q4": sales_q4,
    "Sales_All": sales,
    "Inventory": inv,
}

print("=== ROW COUNTS SUMMARY ===")
for name, df in tables.items():
    print(f"{name:12s} : {len(df):,} rows")

=== ROW COUNTS SUMMARY ===
Products     : 60 rows
Promotions   : 530 rows
Sales_Q1     : 13,545 rows
Sales_Q2     : 13,479 rows
Sales_Q3     : 13,097 rows
Sales_Q4     : 13,165 rows
Sales_All    : 53,286 rows
Inventory    : 2,160 rows


In [55]:
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype

def data_dictionary(df: pd.DataFrame, table_name: str, max_allowed_values=15) -> pd.DataFrame:
    rows = []
    for col in df.columns:
        s = df[col]
        dtype = str(s.dtype)
        non_null = int(s.notna().sum())
        nulls = int(s.isna().sum())
        unique = int(s.nunique(dropna=True))
        # safe sample value
        sample = s.dropna().iloc[0] if non_null else None

        allowed_values = None
        if not is_datetime64_any_dtype(s) and not is_numeric_dtype(s) and unique <= max_allowed_values:
            allowed_values = ", ".join(map(str, sorted(s.dropna().unique())[:max_allowed_values]))

        rows.append({
            "table": table_name,
            "column": col,
            "dtype": dtype,
            "non_null": non_null,
            "nulls": nulls,
            "unique": unique,
            "example_value": sample,
            "allowed_values(sampled)": allowed_values,
            "suggested_description": ""  # fill in plain-English definition later
        })
    return pd.DataFrame(rows).sort_values(["table","column"])
    
# buliding data dictionaries
dd_products   = data_dictionary(products,  "products")
dd_promos     = data_dictionary(promotions,"promotions")
dd_sales      = data_dictionary(sales,     "sales_all")
dd_inventory  = data_dictionary(inv,       "inventory")

# Combine
data_dict = pd.concat([dd_products, dd_promos, dd_sales, dd_inventory], ignore_index=True)

print("Data Dictionary preview (first 25 rows):")
display(data_dict.head(25))

Data Dictionary preview (first 25 rows):


,table,column,dtype,non_null,nulls,unique,example_value,allowed_values(sampled),suggested_description
0,products,brand,object,60,0,21,Rainshadow Produce,None,
1,products,category_name,object,60,0,60,Apples,None,
2,products,department_name,object,60,0,8,Produce,"Bakery, Bulk, Dairy, Deli, Grocery, Meat/Seafo...",
3,products,pack_size,int64,60,0,4,1,None,
4,products,product_name,object,60,0,60,Rainshadow Produce Apples,None,
5,products,product_upc,int64,60,0,60,433218196001,None,
6,products,regular_price,float64,60,0,59,3.11,None,
7,products,size,object,60,0,20,1 lb,None,
8,products,unit,object,60,0,6,lb,"ct, each, gal, lb, oz, qt",
9,products,unit_cost,float64,60,0,57,2.31,None,


In [56]:
# STEP 8 — Generate a Mermaid ERD and save to file

from pathlib import Path

erd_text = """
```mermaid
erDiagram
  PRODUCTS {
    INT product_upc PK
    STRING product_name
    STRING brand
    STRING department_name
    STRING category_name
    STRING size
    STRING unit
    STRING vendor_name
    STRING vendor_phone
    FLOAT regular_price
    FLOAT unit_cost
    INT pack_size
  }

  SALES {
    STRING receipt_id PK
    INT    line_number PK
    DATETIME sale_datetime
    DATE   txn_date
    INT    store_id
    STRING cashier_name
    STRING tender_type
    STRING customer_segment
    INT    product_upc FK
    INT    qty
    FLOAT  unit_price
    FLOAT  line_amount
    FLOAT  unit_price_effective
    FLOAT  line_subtotal
    FLOAT  tax_amount
    BOOLEAN weekend_flag
    INT    promo_id
    STRING promo_type
  }

  INVENTORY_SNAPSHOT {
    DATE   snapshot_date PK
    INT    store_id PK
    INT    product_upc FK PK
    INT    on_hand_qty
    FLOAT  unit_cost
    FLOAT  inventory_cost_value
  }

  PROMOTIONS {
    INT    promo_id PK
    INT    product_upc FK
    STRING promo_type
    INT    discount_percent
    DATE   start_date
    DATE   end_date
  }

  STORE {
    INT    store_id PK
    STRING store_name
    STRING store_address
    STRING store_city
    STRING store_state
    INT    store_zip
  }

  DIM_DATE {
    DATE   d PK
    INT    y
    INT    q
    INT    m
    INT    dow
    BOOLEAN is_holiday
  }

  PRODUCTS ||--o{ SALES : "sold as"
  PRODUCTS ||--o{ INVENTORY_SNAPSHOT : "stocked as"
  PRODUCTS ||--o{ PROMOTIONS : "promoted as"

  PROMOTIONS ||--o{ SALES : "applied to"

  STORE ||--o{ SALES : "has"
  STORE ||--o{ INVENTORY_SNAPSHOT : "tracks"

  DIM_DATE ||--o{ SALES : "occurs on"
  DIM_DATE ||--o{ INVENTORY_SNAPSHOT : "snapshot on"
"""

out_path = Path("erd_mermaid.md")
out_path.write_text(erd_text, encoding="utf-8")

print(f"ERD Markdown file created at: {out_path.resolve()}")
print("\nPreview (first 10 lines):")
print("\n".join(erd_text.splitlines()[:10]))

ERD Markdown file created at: C:\Users\patri\erd_mermaid.md

Preview (first 10 lines):

```mermaid
erDiagram
  PRODUCTS {
    INT product_upc PK
    STRING product_name
    STRING brand
    STRING department_name
    STRING category_name
    STRING size


In [57]:
from pathlib import Path

ddl = """
-- Postgres DDL generated from the ERD
-- Tables: products, stores, dim_date, promotions, sales_txn, inventory_snapshot

BEGIN;

CREATE TABLE IF NOT EXISTS products (
  product_upc        BIGINT PRIMARY KEY,
  product_name       TEXT NOT NULL,
  brand              TEXT,
  department_name    TEXT,
  category_name      TEXT,
  size               TEXT,
  unit               TEXT,
  vendor_name        TEXT,
  vendor_phone       TEXT,
  regular_price      NUMERIC(10,2),
  unit_cost          NUMERIC(10,2),
  pack_size          INTEGER CHECK (pack_size IS NULL OR pack_size >= 0)
);

CREATE TABLE IF NOT EXISTS stores (
  store_id       INTEGER PRIMARY KEY,
  store_name     TEXT,
  store_address  TEXT,
  store_city     TEXT,
  store_state    VARCHAR(2),
  store_zip      INTEGER
);

CREATE TABLE IF NOT EXISTS dim_date (
  d           DATE PRIMARY KEY,
  y           INTEGER,
  q           INTEGER CHECK (q BETWEEN 1 AND 4),
  m           INTEGER CHECK (m BETWEEN 1 AND 12),
  dow         INTEGER CHECK (dow BETWEEN 0 AND 6),
  is_holiday  BOOLEAN
);

CREATE TABLE IF NOT EXISTS promotions (
  promo_id          INTEGER PRIMARY KEY,
  product_upc       BIGINT REFERENCES products(product_upc),
  promo_type        TEXT,
  discount_percent  NUMERIC(5,2) CHECK (discount_percent BETWEEN 0 AND 100),
  start_date        DATE NOT NULL,
  end_date          DATE NOT NULL,
  CHECK (start_date <= end_date)
);

CREATE TABLE IF NOT EXISTS sales_txn (
  receipt_id            TEXT NOT NULL,
  line_number           INTEGER NOT NULL,
  sale_datetime         TIMESTAMP,
  txn_date              DATE NOT NULL,
  store_id              INTEGER REFERENCES stores(store_id),
  cashier_name          TEXT,
  tender_type           TEXT,
  customer_segment      TEXT,
  product_upc           BIGINT REFERENCES products(product_upc),
  qty                   INTEGER NOT NULL CHECK (qty >= 0),
  unit_price            NUMERIC(10,2),
  line_amount           NUMERIC(12,2),
  unit_price_effective  NUMERIC(10,2),
  line_subtotal         NUMERIC(12,2),
  tax_amount            NUMERIC(12,2),
  weekend_flag          BOOLEAN,
  promo_id              INTEGER REFERENCES promotions(promo_id),
  promo_type            TEXT,
  PRIMARY KEY (receipt_id, line_number)
);

CREATE TABLE IF NOT EXISTS inventory_snapshot (
  snapshot_date         DATE NOT NULL,
  store_id              INTEGER NOT NULL REFERENCES stores(store_id),
  product_upc           BIGINT  NOT NULL REFERENCES products(product_upc),
  on_hand_qty           INTEGER CHECK (on_hand_qty IS NULL OR on_hand_qty >= 0),
  unit_cost             NUMERIC(10,2),
  inventory_cost_value  NUMERIC(12,2),
  PRIMARY KEY (snapshot_date, store_id, product_upc)
);

-- Helpful analytics / join indexes
CREATE INDEX IF NOT EXISTS idx_sales_txn_date       ON sales_txn (txn_date);
CREATE INDEX IF NOT EXISTS idx_sales_store          ON sales_txn (store_id);
CREATE INDEX IF NOT EXISTS idx_sales_product        ON sales_txn (product_upc);
CREATE INDEX IF NOT EXISTS idx_sales_store_date     ON sales_txn (store_id, txn_date);

CREATE INDEX IF NOT EXISTS idx_promos_product_range ON promotions (product_upc, start_date, end_date);

CREATE INDEX IF NOT EXISTS idx_inv_store_prod_date  ON inventory_snapshot (store_id, product_upc, snapshot_date);

COMMIT;
"""

out = Path("ddl_postgres.sql")
out.write_text(ddl, encoding="utf-8")
print(f"Wrote normalized schema DDL to: {out.resolve()}")

Wrote normalized schema DDL to: C:\Users\patri\ddl_postgres.sql
